In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

# Define a list to store all the route data
all_data = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to list
            all_data.append({'routename': routename, 'routelink': routelink})

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df = pd.DataFrame(all_data)

# Print the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('scraped_routes.csv', index=False)

# Close the driver
driver.quit()

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
                                  routename  \
0                   Hyderabad to Vijayawada   
1                   Vijayawada to Hyderabad   
2                       Hyderabad to Ongole   
3                 Kakinada to Visakhapatnam   
4                     Bangalore to Tirupati   
5                       Bangalore to Kadapa   
6                       Ongole to Hyderabad   
7                       Kadapa to Bangalore   
8    Chittoor (Andhra Pradesh) to Bangalore   
9                 Visakhapatnam to Kakinada   
10  Bangalore to Anantapur (andhra pradesh)   
11   Bangalore to Chittoor (Andhra Pradesh)   
12  Anantapur (andhra pradesh) to Bangalore   
13                     Hyderabad to Kurnool   
14                    Tirupati to Bangalore   
15                Narasaraopet to Hydera

In [3]:
#retrive the bus details
driver = webdriver.Chrome()
Bus_names_A= []
Bus_types_A = []
Start_Time_A = []
End_Time_A = []
Ratings_A = []
Total_Duration_A = []
Prices_A = []
Seats_Available_A = []
Route_names = []
Route_links = []

for i,r in df_1.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_A.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_A.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_A.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_A.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_A.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_A.append(ratings.text)
    for price_elem in price:
        Prices_A.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_A.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [5]:
data_1 = {
    'Bus_name':  Bus_names_A,
    'Bus_type': Bus_types_A,
    'Start_time': Start_Time_A,
    'End_time': End_Time_A,
    'Total_duration': Total_Duration_A,
    'Price': Prices_A,
    "Seats_Available":Seats_Available_A,
    "Ratings":Ratings_A,
    'Route_link': Route_links,
    'Route_name':Route_names
}

df_buses_1 = pd.DataFrame(data_1)
df_buses_1.to_csv('buses_data1.csv', index=False)
df_buses_1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name


In [46]:
# Initialize the Chrome driver
driver_k1 = webdriver.Chrome()
wait = WebDriverWait(driver_k1, 30)  # Increased timeout

# Navigate to the website
driver_k1.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

# Define lists to store all the route data
route_k1= []
routelink_k1 = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_k1.append(routename)
            routelink_k1.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 3):
    scrape_page()
    if page_number < 2:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_k1)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_2 = pd.DataFrame({
    'Route Name': route_k1,
    'Route Link': routelink_k1
})

# Print the DataFrame
print(df_2)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
                         Route Name  \
0            Bangalore to Kozhikode   
1            Kozhikode to Bangalore   
2            Kozhikode to Ernakulam   
3            Ernakulam to Kozhikode   
4               Bangalore to Kannur   
5               Kozhikode to Mysore   
6               Kannur to Bangalore   
7   Kozhikode to Thiruvananthapuram   
8               Mysore to Kozhikode   
9    Bangalore to Kalpetta (kerala)   
10   Kalpetta (kerala) to Bangalore   
11  Thiruvananthapuram to Kozhikode   
12            Kozhikode to Thrissur   
13            Kozhikode to Kottayam   
14            Kottayam to Kozhikode   

                                           Route Link  
0   https://www.redbus.in/bus-tickets/bangalore-to...  
1   https://www.redbus.in/bus-tickets/kozhikode-to...  
2   https://www.redbus.in/bus-tickets/kozhikode-to...  
3   https://www.redbus.in/bus-tickets/ernakulam-to...  
4   https://www.redbus.in/bus-tickets/banga

In [48]:
#retrive the bus details
driver_k1= webdriver.Chrome()
Bus_names_k1 = []
Bus_types_k1 = []
Start_Time_k1= []
End_Time_k1 = []
Ratings_k1= []
Total_Duration_k1 = []
Prices_k1 = []
Seats_Available_k1 = []
Route_names = []
Route_links = []

for i,r in df_2.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_k1.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k1.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k1.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k1.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k1).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k1.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k1.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k1.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k1.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k1.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k1.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k1.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k1.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k1.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k1.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k1.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k1.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k1.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k1.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k1.append(ratings.text)
    for price_elem in price:
        Prices_k1.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k1.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [50]:
data_2= {
    'Bus_name':  Bus_names_k1,
    'Bus_type': Bus_types_k1,
    'Start_time': Start_Time_k1,
    'End_time': End_Time_k1,
    'Total_duration': Total_Duration_k1,
    'Price': Prices_k1,
    "Seats_Available":Seats_Available_k1,
    "Ratings":Ratings_k1,
    'Route_link': Route_links,
    'Route_name':Route_names
}

df_buses_2= pd.DataFrame(data_2)
df_buses_2.to_csv('buses_data2.csv', index=False)
df_buses_2

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,INR 729,1 Seat available,2.0\n29,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:02,00:50,08h 48m,INR 567,5 Seats available,2.4\n21,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,KSRTC (Kerala) - 171,Swift Deluxe Non AC Air Bus (2+2),16:16,01:20,09h 04m,INR 640,8 Seats available,4.1\n9,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,01:25,08h 40m,INR 640,2 Seats available,4.1\n12,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,04:24,09h 20m,INR 640,23 Seats available,4.2\n44,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
132,Sonia Travels,A/C Semi Sleeper (2+2),18:00,02:20,08h 20m,INR 750,34 Seats available,4.0,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
133,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,02:29,07h 19m,INR 1100,3 Seats available,4.0,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
134,Madhavi Travels,A/C Sleeper (2+1),19:00,03:00,08h 00m,INR 1399,1 Seat available,3.9,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
135,Madhavi Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),23:59,04:30,04h 31m,INR 1499,38 Seats available,3.9,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode


In [21]:
# Initialize the Chrome driver
driver_WB = webdriver.Chrome()
wait = WebDriverWait(driver_WB, 30)  # Increased timeout

# Navigate to the website
driver_WB.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")

# Define lists to store all the route data
route_WB= []
routelink_WB = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_WB.append(routename)
            routelink_WB.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 5):
    scrape_page()
    if page_number < 4:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_WB)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_3 = pd.DataFrame({
    'Route Name': route_WB,
    'Route Link': routelink_WB
})

# Print the DataFrame
print(df_3)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
                                           Route Name  \
0                      Digha to Barasat (West Bengal)   
1                                Durgapur to Calcutta   
2                                   Digha to Calcutta   
3                                    Kolkata to Digha   
4                      Barasat (West Bengal) to Digha   
5                                     Kolkata to Suri   
6                  Barasat (West Bengal) to Midnapore   
7                                Midnapore to Kolkata   
8                   Barasat (West Bengal) to Kolaghat   
9            Barasat (West Bengal) to Contai (Kanthi)   
10                                     Habra to Digha   
11  Barasat (West Bengal) to Nandakumar (west bengal)   
12                  Kolkata to Durgapur (West Bengal)   
13                                     Digha t

In [23]:
# retrive the bus details
driver_WB = webdriver.Chrome()
Bus_names_WB = []
Bus_types_WB = []
Start_Time_WB = []
End_Time_WB = []
Ratings_WB = []
Total_Duration_WB = []
Prices_WB = []
Seats_Available_WB = []
Route_names = []
Route_links = []

for i,r in df_2.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]
# Loop through each link
    driver_WB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_WB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_WB.page_source
        
        ActionChains(driver_WB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_WB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_WB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_WB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_WB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_WB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_WB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_WB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_WB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_WB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_WB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_WB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_WB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_WB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_WB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_WB.append(ratings.text)
    for price_elem in price:
        Prices_WB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_WB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [25]:
# from list to convert data frame
data_3 = {
    'Bus_name': Bus_names_WB,
    'Bus_type': Bus_types_WB,
    'Start_time': Start_Time_WB,
    'End_time': End_Time_WB,
    'Total_duration': Total_Duration_WB,
    'Price': Prices_WB,
    "Seats_Available":Seats_Available_WB,
    "Ratings":Ratings_WB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_3= pd.DataFrame(data_3)
df_buses_3.to_csv('buses_data3.csv', index=False)
df_buses_3

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Ashray Grand,Volvo Multi-Axle A/C Sleeper (2+1),21:00,10:15,13h 15m,INR 1320,8 Seats available,4.2\n516,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,Gogte Anand Travels,Volvo Multi-Axle I-Shift A/C Sleeper (2+1),19:30,07:55,12h 25m,INR 799,9 Seats available,4.1\n90,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),19:30,08:05,12h 35m,INR 551,3 Seats available,4.3\n395,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,IntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),21:15,08:45,11h 30m,INR 900,6 Seats available,4.2\n230,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Atmaram Gobus,VE A/C Sleeper (2+1),19:25,07:30,12h 05m,INR 808,5 Seats available,4.6\n595,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
98,Laxmi Queen Prime,A/C Sleeper (2+1),19:30,05:00,09h 30m,INR 899,10 Seats available,4.4\n8,https://www.redbus.in/bus-tickets/goa-to-satara,Goa to Satara
99,Sangitam Travels Tanzanite,A/C Sleeper (2+1),21:00,04:30,07h 30m,INR 905,13 Seats available,4.2\n9,https://www.redbus.in/bus-tickets/goa-to-satara,Goa to Satara
100,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+1),20:00,20:30,00h 30m,INR 150,15 Seats available,3.7,https://www.redbus.in/bus-tickets/mopa-airport...,Mopa Airport to Margao
101,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),21:00,23:00,02h 00m,INR 400,47 Seats available,4.9,https://www.redbus.in/bus-tickets/mopa-airport...,Mopa Airport to Margao


In [27]:
# Initialize the Chrome driver
driver_C = webdriver.Chrome()
wait = WebDriverWait(driver_C, 30)  # Increased timeout

# Navigate to the website
driver_C.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")

# Define lists to store all the route data
route_C= []
routelink_C = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_C.append(routename)
            routelink_C.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_C)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_4= pd.DataFrame({
    'Route Name': route_C,
    'Route Link': routelink_C
})

# Print the DataFrame
print(df_4)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
                                      Route Name  \
0                     Yamuna Nagar to Chandigarh   
1                            Chandigarh to Delhi   
2                            Delhi to Chandigarh   
3                         Ludhiana to Chandigarh   
4                     Chandigarh to Yamuna Nagar   
5                         Chandigarh to Ludhiana   
6      Hamirpur (Himachal Pradesh) to Chandigarh   
7                        Chandigarh to Vrindavan   
8      Chandigarh to Hamirpur (Himachal Pradesh)   
9                        Chandigarh to Pathankot   
10                        Chandigarh to Dehradun   
11                        Dehradun to Chandigarh   
12                       Pathankot to Chandigarh   
13     Sujanpur (himachal pradesh) to Chandigarh   
14     Chand

In [29]:
# retrive the bus details
driver_C = webdriver.Chrome()
Bus_names_C = []
Bus_types_C = []
Start_Time_C = []
End_Time_C = []
Ratings_C = []
Total_Duration_C = []
Prices_C = []
Seats_Available_C = []
Route_names = []
Route_links = []

for i,r in df_4.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]
# Loop through each link
    driver_C.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_C.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_C.page_source
        
        ActionChains(driver_C).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_C.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_C.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_C.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_C.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_C.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_C.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_C.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_C.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_C.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_C.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_C.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_C.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_C.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_C.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_C.append(ratings.text)
    for price_elem in price:
        Prices_C.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_C.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [32]:
# from list to convert data frame
data_4 = {
    'Bus_name': Bus_names_C,
    'Bus_type': Bus_types_C,
    'Start_time': Start_Time_C,
    'End_time': End_Time_C,
    'Total_duration': Total_Duration_C,
    'Price': Prices_C,
    "Seats_Available":Seats_Available_C,
    "Ratings":Ratings_C,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_4= pd.DataFrame(data_4)
df_buses_4.to_csv('buses_data4.csv', index=False)
df_buses_4

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,23:30,02h 50m,INR 599,23 Seats available,3.8,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
1,City Land Travels,A/C Sleeper (2+1),22:20,00:20,02h 00m,513,27 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
2,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:00,23:30,02h 30m,INR 599,38 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
3,RSRTC - 192348,Super Luxury Volvo AC Seater Pushback 2+2,21:15,02:00,04h 45m,INR 691,26 Seats available,3.2\n5,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi
4,Khurana Bus Service,Volvo A/C (2+2),21:10,02:40,05h 30m,300,30 Seats available,3.6\n48,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi
...,...,...,...,...,...,...,...,...,...,...
155,RAO TRAVELS INDIA PVT LTD.,Mercedes Benz A/C (2+2),23:00,03:30,04h 30m,INR 3000,48 Seats available,2.2,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jammu (j and k)
156,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),23:30,03:30,04h 00m,INR 666,40 Seats available,3.5\n13,https://www.redbus.in/bus-tickets/ferozpur-to-...,Firozpur to Chandigarh
157,Shree Mahaveer Travels Jaipur,VE A/C Seater / Sleeper (2+1),22:00,07:55,09h 55m,854,3 Seats available,4.1\n120,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani
158,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,07:30,10h 00m,INR 849,7 Seats available,4.6\n16,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Haldwani


In [4]:
# Initialize the Chrome driver
driver_AS = webdriver.Chrome()
wait = WebDriverWait(driver_AS, 30)  # Increased timeout

# Navigate to the website
driver_AS.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

# Define lists to store all the route data
route_AS= []
routelink_AS = []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_AS.append(routename)
            routelink_AS.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_AS)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_5= pd.DataFrame({
    'Route Name': route_AS,
    'Route Link': routelink_AS
})

# Print the DataFrame
print(df_5)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
                             Route Name  \
0                    Tezpur to Guwahati   
1                    Guwahati to Tezpur   
2            Guwahati to Nagaon (Assam)   
3            Nagaon (Assam) to Guwahati   
4                  Goalpara to Guwahati   
5             Jorhat to North Lakhimpur   
6                    Dhubri to Guwahati   
7                   Jorhat to Dibrugarh   
8             North Lakhimpur to Jorhat   
9           North Lakhimpur to Sibsagar   
10                   Guwahati to Dhubri   
11               Dhekiajuli to Guwahati   
12          Sibsagar to North Lakhimpur   
13                  Dibrugarh to Jorhat   
14                    Jorhat to Dhemaji   
15                   Jorhat to Tinsukia   
16                  Tezpur to Dibrugarh   
17                

In [10]:
# retrive the bus details
driver_AS = webdriver.Chrome()
Bus_names_AS = []
Bus_types_AS = []
Start_Time_AS = []
End_Time_AS = []
Ratings_AS = []
Total_Duration_AS = []
Prices_AS = []
Seats_Available_AS = []
Route_names = []
Route_links = []

for i,r in df_5.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]
# Loop through each link
    driver_AS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_AS.page_source
        
        ActionChains(driver_AS).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_AS.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_AS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AS.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AS.append(ratings.text)
    for price_elem in price:
        Prices_AS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AS.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [12]:
# from list to convert data frame
data_5= {
    'Bus_name': Bus_names_AS,
    'Bus_type': Bus_types_AS,
    'Start_time': Start_Time_AS,
    'End_time': End_Time_AS,
    'Total_duration': Total_Duration_AS,
    'Price': Prices_AS,
    "Seats_Available":Seats_Available_AS,
    "Ratings":Ratings_AS,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_5= pd.DataFrame(data_5)
df_buses_5.to_csv('buses_data5.csv', index=False)
df_buses_5

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,03:45,05h 30m,450,19 Seats available,3.6\n27,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04:00,04h 20m,INR 500,31 Seats available,1.7\n17,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
2,Yatra Travels,NON A/C Seater (2+1),18:45,23:30,04h 45m,350,8 Seats available,3.7\n24,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
3,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,00:00,04h 00m,350,10 Seats available,3.7\n33,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
4,KIRAN TRAVELS,A/C Seater (2+1),20:40,00:30,03h 50m,500,11 Seats available,3.1\n6,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
...,...,...,...,...,...,...,...,...,...,...
97,RATNAGIRI TRANSPORT,Bharat Benz A/C Seater /Sleeper (2+1),21:31,04:30,06h 59m,INR 600,14 Seats available,4.0\n94,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Golaghat
98,NAMBOR TRANSPORT,Bharat Benz A/C Seater (2+1),21:00,04:00,07h 00m,INR 600,16 Seats available,4.1\n269,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Golaghat
99,NAMBOR TRANSPORT,NON A/C Seater (2+1),20:30,03:15,06h 45m,INR 500,20 Seats available,3.5\n99,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Golaghat
100,ZAMZAM TRAVELS,Bharat Benz A/C Seater (2+1),20:15,03:20,07h 05m,INR 550,26 Seats available,3.7\n82,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Golaghat


In [14]:
# Initialize the Chrome driver
driver_H = webdriver.Chrome()
wait = WebDriverWait(driver_H, 30)  # Increased timeout

# Navigate to the website
driver_H.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

# Define lists to store all the route data
route_H= []
routelink_H= []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_H.append(routename)
            routelink_H.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 5):
    scrape_page()
    if page_number < 4:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_H)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_6= pd.DataFrame({
    'Route Name': route_H,
    'Route Link': routelink_H
})

# Print the DataFrame
print(df_6)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
                                      Route Name  \
0                                Delhi to Shimla   
1      Chandigarh to Hamirpur (Himachal Pradesh)   
2      Hamirpur (Himachal Pradesh) to Chandigarh   
3                                Shimla to Delhi   
4                            Delhi to Chandigarh   
5           Hamirpur (Himachal Pradesh) to Delhi   
6        Chamba (Himachal Pradesh) to Chandigarh   
7           Delhi to Hamirpur (Himachal Pradesh)   
8   Chandigarh to Dharamshala (Himachal Pradesh)   
9             Delhi to Chamba (Himachal Pradesh)   
10            Chamba (Himachal Pradesh) to Delhi   
11                          Kangra to Chandigarh   
12                          Shimla to Chandigarh   
13             Delhi to Baddi (Himachal Pradesh)   
14                                Delhi to Solan   
15          

In [16]:
# retrive the bus details
driver_H = webdriver.Chrome()
Bus_names_H = []
Bus_types_H = []
Start_Time_H = []
End_Time_H = []
Ratings_H = []
Total_Duration_H = []
Prices_H = []
Seats_Available_H = []
Route_names_H = []
Route_links_H = []
for i,r in df_6.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_H.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_H.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
         
    scrolling = True
    while scrolling:
        old_page_source = driver_H.page_source
        
        ActionChains(driver_H).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_H.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_H.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_H.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_H.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_H.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_H.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_H.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_H.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_H.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_H.append(bus.text)
        Route_links_H.append(link)
        Route_names_H.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_H.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_H.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_H.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_H.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_H.append(ratings.text)
    for price_elem in price:
        Prices_H.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_H.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [18]:
# from list to convert data frame
data_6 = {
    'Bus_name': Bus_names_H,
    'Bus_type': Bus_types_H,
    'Start_time': Start_Time_H,
    'End_time': End_Time_H,
    'Total_duration': Total_Duration_H,
    'Price': Prices_H,
    "Seats_Available":Seats_Available_H,
    "Ratings":Ratings_H,
    'Route_link': Route_links_H,
    'Route_name': Route_names_H
}

df_buses_6= pd.DataFrame(data_6)
df_buses_6.to_csv('buses_data6.csv', index=False)
df_buses_6

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:50,07:40,09h 50m,750,8 Seats available,4.2\n341,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),22:50,07:40,08h 50m,759,3 Seats available,4.2\n313,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,Bedi Travels,Volvo A/C Semi Sleeper (2+2),22:30,08:00,09h 30m,521,3 Seats available,4.8\n796,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,Zingbus Plus,Bharat Benz A/C Semi Sleeper (2+2),22:05,09:05,11h 00m,854,5 Seats available,4.4\n462,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,Laxmi holidays,Volvo Multi-Axle A/C Semi Sleeper (2+2),23:25,07:45,08h 20m,759,4 Seats available,4.5\n671,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
176,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),21:30,07:30,10h 00m,854,15 Seats available,4.5\n783,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
177,Reo India Travels,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),19:40,06:05,10h 25m,475,10 Seats available,4.2\n29,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
178,Zingbus Plus,Volvo 9600 A/C Semi Sleeper (2+2),21:00,08:40,11h 40m,726,3 Seats available,4.8\n200,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
179,Zingbus Plus,Volvo 9600 A/C Semi Sleeper (2+2),20:30,07:45,11h 15m,622,5 Seats available,4.4\n212,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi


In [20]:
# Initialize the Chrome driver
driver_SB = webdriver.Chrome()
wait = WebDriverWait(driver_SB, 30)  # Increased timeout

# Navigate to the website
driver_SB.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")

# Define lists to store all the route data
route_SB= []
routelink_SB= []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_SB.append(routename)
            routelink_SB.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_SB)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_7= pd.DataFrame({
    'Route Name': route_SB,
    'Route Link': routelink_SB
})

# Print the DataFrame
print(df_7)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
                                           Route Name  \
0                                Durgapur to Calcutta   
1                                  Kolkata to Burdwan   
2                                  Haldia to Calcutta   
3                                   Kolkata to Haldia   
4                   Kolkata to Durgapur (West Bengal)   
5                   Kolkata to Arambagh (West Bengal)   
6                                Midnapore to Kolkata   
7                                    Kolkata to Digha   
8                                   Digha to Calcutta   
9                                  Kolkata to Bankura   
10                               Kolkata to Midnapore   
11                   Kolkata to Asansol (West Bengal)   
12                                Kolkata to Nimtou

In [22]:
# retrive the bus details
driver_SB = webdriver.Chrome()
Bus_names_SB = []
Bus_types_SB = []
Start_Time_SB = []
End_Time_SB = []
Ratings_SB = []
Total_Duration_SB = []
Prices_SB = []
Seats_Available_SB = []
Route_names = []
Route_links = []

for i,r in df_7.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_SB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_SB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_SB.page_source
        
        ActionChains(driver_SB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_SB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_SB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_SB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_SB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_SB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_SB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_SB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_SB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_SB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_SB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_SB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_SB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_SB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_SB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_SB.append(ratings.text)
    for price_elem in price:
        Prices_SB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_SB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [24]:
# from list to convert data frame
data_7 = {
    'Bus_name': Bus_names_SB,
    'Bus_type': Bus_types_SB,
    'Start_time': Start_Time_SB,
    'End_time': End_Time_SB,
    'Total_duration': Total_Duration_SB,
    'Price': Prices_SB,
    "Seats_Available":Seats_Available_SB,
    "Ratings":Ratings_SB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_7= pd.DataFrame(data_7)
df_buses_7.to_csv('buses_data7.csv', index=False)
df_buses_7

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,SBSTC-KOLKATA - BURDWAN - 20:00 (BWN DEPOT) - 103,Non AC Seater (2+3),20:00,22:00,02h 00m,INR 95,33 Seats available,3.2\n33,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
1,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),19:30,20:40,01h 20m,INR 429,15 Seats available,4.0\n17,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
2,Suman sabir,Non AC Seater (2+3),21:00,23:59,02h 59m,INR 111,54 Seats available,1.2,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
3,Pammi Travels,A/C Sleeper (2+1),21:00,23:59,02h 59m,INR 1300,12 Seats available,3.7,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
4,Pammi Travels,Non A/C Seater / Sleeper (2+1),21:00,23:59,02h 59m,INR 650,20 Seats available,3.7,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Burdwan
...,...,...,...,...,...,...,...,...,...,...
58,Basu Travels,A/C Seater Push Back (2+3),23:30,03:45,04h 15m,INR 255,20 Seats available,2.7,https://www.redbus.in/bus-tickets/kolkata-to-r...,Kolkata to Ramnagar (West Bengal)
59,Bengal Surface Transport,A/C Seater / Sleeper (2+2),23:55,04:40,04h 45m,INR 300,37 Seats available,2.9,https://www.redbus.in/bus-tickets/kolkata-to-r...,Kolkata to Ramnagar (West Bengal)
60,Basu Travels,A/C Seater / Sleeper (2+2),23:59,04:40,04h 41m,INR 300,34 Seats available,2.7,https://www.redbus.in/bus-tickets/kolkata-to-r...,Kolkata to Ramnagar (West Bengal)
61,Bengal Surface Transport,A/C Seater Push Back (2+3),23:59,04:45,04h 46m,INR 399,20 Seats available,2.9,https://www.redbus.in/bus-tickets/kolkata-to-r...,Kolkata to Ramnagar (West Bengal)


In [26]:
# Initialize the Chrome driver
driver_P = webdriver.Chrome()
wait = WebDriverWait(driver_P, 30)  # Increased timeout

# Navigate to the website
driver_P.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")

# Define lists to store all the route data
route_P= []
routelink_P= []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_P.append(routename)
            routelink_P.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 3):
    scrape_page()
    if page_number < 2:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_P)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_8= pd.DataFrame({
    'Route Name': route_P,
    'Route Link': routelink_P
})

# Print the DataFrame
print(df_8)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
                    Route Name  \
0             Delhi to Patiala   
1             Patiala to Delhi   
2            Delhi to Ludhiana   
3            Ludhiana to Delhi   
4           Jalandhar to Delhi   
5           Delhi to Jalandhar   
6     Delhi Airport to Patiala   
7    Ludhiana to Delhi Airport   
8    Delhi Airport to Ludhiana   
9   Jalandhar to Delhi Airport   
10   Phagwara to Delhi Airport   
11           Delhi to Phagwara   
12  Delhi Airport to Jalandhar   
13           Phagwara to Delhi   
14           Delhi to Amritsar   
15           Amritsar to Delhi   
16   Amritsar to Delhi Airport   
17   Delhi Airport to Phagwara   

                                           Route Link  
0   https://www.redbus.in/bus-tickets/delhi-to-pat...  
1   https://www.redbus.in/bus-tickets/patiala-to-d...  
2   https://www.redbus.in/bus-tickets/delhi-to-lud...  
3   https://www.redbus.in/bus-tickets/ludhiana-to-...  
4   https://www.redbu

In [28]:
# retrive the bus details
driver_P = webdriver.Chrome()
Bus_names_P = []
Bus_types_P = []
Start_Time_P = []
End_Time_P = []
Ratings_P = []
Total_Duration_P = []
Prices_P = []
Seats_Available_P = []
Route_names = []
Route_links = []

for i,r in df_8.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_P.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_P.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_P.page_source
        
        ActionChains(driver_P).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_P.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_P.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_P.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_P.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_P.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_P.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_P.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_P.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_P.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_P.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_P.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_P.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_P.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_P.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_P.append(ratings.text)
    for price_elem in price:
        Prices_P.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_P.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [30]:
# from list to convert data frame
data_8 = {
    'Bus_name': Bus_names_P,
    'Bus_type': Bus_types_P,
    'Start_time': Start_Time_P,
    'End_time': End_Time_P,
    'Total_duration': Total_Duration_P,
    'Price': Prices_P,
    "Seats_Available":Seats_Available_P,
    "Ratings":Ratings_P,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_8= pd.DataFrame(data_8)
df_buses_8.to_csv('buses_data8.csv', index=False)
df_buses_8

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:31,04:00,05h 29m,INR 454,32 Seats available,3.3\n240,https://www.redbus.in/bus-tickets/delhi-to-pat...,Delhi to Patiala
1,PEPSU (Punjab) - 67104,AC Seater Hvac 2+2,22:30,03:31,05h 01m,INR 454,31 Seats available,3.1\n169,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi
2,PEPSU (Punjab) - 67121,Volvo AC Seater (2+2),21:50,04:50,07h 00m,INR 873,38 Seats available,4.0\n4,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
3,RSRTC - 156675,Express Non AC Seater 2+3,22:30,05:15,06h 45m,INR 422,36 Seats available,2.2\n6,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
4,Yolo Bus,VE A/C Seater / Sleeper (2+1),23:30,05:24,05h 54m,INR 600,9 Seats available,4.6\n380,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
...,...,...,...,...,...,...,...,...,...,...
110,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),23:30,08:45,09h 15m,INR 3300,51 Seats available,4.6,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Phagwara
111,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,04:45,08h 15m,INR 2500,51 Seats available,4.6,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Phagwara
112,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),23:30,07:45,08h 15m,INR 2500,44 Seats available,4.6,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Phagwara
113,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),23:30,07:45,08h 15m,INR 2500,44 Seats available,4.6,https://www.redbus.in/bus-tickets/delhi-airpor...,Delhi Airport to Phagwara


In [32]:
# Initialize the Chrome driver
driver_NB = webdriver.Chrome()
wait = WebDriverWait(driver_NB, 30)  # Increased timeout

# Navigate to the website
driver_NB.get("https://www.redbus.in/online-booking/north-bengal-state-transport-corporation")

# Define lists to store all the route data
route_NB= []
routelink_NB= []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_NB.append(routename)
            routelink_NB.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_NB)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_9= pd.DataFrame({
    'Route Name': route_NB,
    'Route Link': routelink_NB
})

# Print the DataFrame
print(df_9)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
                                           Route Name  \
0                              Siliguri to Darjeeling   
1                                 Siliguri to Kolkata   
2                                 Kolkata to Siliguri   
3                                  Raiganj to Kolkata   
4                                  Kolkata to Raiganj   
5                                    Kolkata to Malda   
6                                    Malda to Kolkata   
7   Cooch Behar (West Bengal) to Berhampore (West ...   
8                Kolkata to Cooch Behar (West Bengal)   
9                                Kolkata to Balurghat   
10  Berhampore (West Bengal) to Cooch Behar (West ...   
11               Berhampore (West Bengal) to Siliguri   
12               Cooch Behar (West Bengal) to Raiga

In [34]:
# retrive the bus details
driver_NB = webdriver.Chrome()
Bus_names_NB = []
Bus_types_NB = []
Start_Time_NB = []
End_Time_NB = []
Ratings_NB = []
Total_Duration_NB = []
Prices_NB = []
Seats_Available_NB = []
Route_names = []
Route_links = []

for i,r in df_9.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_NB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_NB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_NB.page_source
        
        ActionChains(driver_NB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_NB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_NB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_NB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_NB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_NB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_NB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_NB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_NB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_NB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_NB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_NB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_NB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_NB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_NB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_NB.append(ratings.text)
    for price_elem in price:
        Prices_NB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_NB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [36]:
# from list to convert data frame
data_9 = {
    'Bus_name': Bus_names_NB,
    'Bus_type': Bus_types_NB,
    'Start_time': Start_Time_NB,
    'End_time': End_Time_NB,
    'Total_duration': Total_Duration_NB,
    'Price': Prices_NB,
    "Seats_Available":Seats_Available_NB,
    "Ratings":Ratings_NB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_9= pd.DataFrame(data_9)
df_buses_9.to_csv('buses_data9.csv', index=False)
df_buses_9

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,INR 105,25 Seats available,3.5\n253,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Darjeeling
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,INR 105,10 Seats available,4.0\n282,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Darjeeling
2,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,07:25,12h 55m,INR 1500,18 Seats available,4.7\n643,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Kolkata
3,Shyamoli Paribahan Pvt Ltd,Volvo Multi-Axle A/C seater/sleeper (2+1),19:30,08:20,12h 50m,INR 1200,20 Seats available,4.5\n727,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Kolkata
4,Shyamoli Paribahan Pvt Ltd,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:00,06:05,12h 05m,INR 1500,21 Seats available,4.5\n423,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Kolkata
...,...,...,...,...,...,...,...,...,...,...
245,Lokenath Bus Service,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),18:15,22:30,04h 15m,INR 650,46 Seats available,4.3,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Raiganj
246,Shree Krishna Travels,NON A/C Seater / Sleeper (2+2),20:00,02:00,06h 00m,INR 250,23 Seats available,3.4,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Raiganj
247,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),14:30,19:15,04h 45m,INR 700,33 Seats available,2.4,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Raiganj
248,Lokenath (Maa Tara),NON A/C Seater / Sleeper (2+2),15:15,20:15,05h 00m,INR 280,11 Seats available,2.9,https://www.redbus.in/bus-tickets/siliguri-to-...,Siliguri to Raiganj


In [38]:

# Initialize the Chrome driver
driver_KA = webdriver.Chrome()
wait = WebDriverWait(driver_KA, 30)  # Increased timeout

# Navigate to the website
driver_KA.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")

# Define lists to store all the route data
route_KA= []
routelink_KA= []

def scrape_page():
    # Locate elements (container)
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routescontainer:
        try:
            routename = route.find_element(By.CLASS_NAME, 'route').text
            routelink = route.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            # Append extracted data to individual lists
            route_KA.append(routename)
            routelink_KA.append(routelink)

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 5):
    scrape_page()
    if page_number < 4:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver_KA)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Create a DataFrame from the collected data
df_10= pd.DataFrame({
    'Route Name': route_KA,
    'Route Link': routelink_KA
})

# Print the DataFrame
print(df_10)

# Optionally, save the DataFrame to

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
                         Route Name  \
0                       Pune to Goa   
1                       Goa to Pune   
2                     Mumbai to Goa   
3                  Bangalore to Goa   
4                  Goa to Bangalore   
5                     Goa to Mumbai   
6                 Pandharpur to Goa   
7                 Goa to Pandharpur   
8                    Solapur to Goa   
9    Calangute (goa) to Goa Airport   
10     Goa to Kolhapur(Maharashtra)   
11                   Goa to Solapur   
12                    Goa to Sangli   
13   Goa Airport to Calangute (goa)   
14         Goa to Sangola (Solapur)   
15         Sangola (Solapur) to Goa   
16  Mopa Airport to Calangute (goa)   
17  Calangute (goa) to Mopa Airport   
18              Goa to Mopa Airport   
19                     Goa to Miraj   
20            Panaji to G

In [40]:
# retrive the bus details
driver_KA = webdriver.Chrome()
Bus_names_KA = []
Bus_types_KA = []
Start_Time_KA = []
End_Time_KA = []
Ratings_KA = []
Total_Duration_KA = []
Prices_KA = []
Seats_Available_KA = []
Route_names = []
Route_links = []

for i,r in df_10.iterrows():
    link=r["Route Link"]
    routes=r["Route Name"]

# Loop through each link
    driver_KA.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_KA.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_KA.page_source
        
        ActionChains(driver_KA).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_KA.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_KA.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_KA.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_KA.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_KA.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_KA.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_KA.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_KA.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_KA.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_KA.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_KA.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_KA.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_KA.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_KA.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_KA.append(ratings.text)
    for price_elem in price:
        Prices_KA.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_KA.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [42]:
# from list to convert data frame
data_10 = {
    'Bus_name': Bus_names_KA,
    'Bus_type': Bus_types_KA,
    'Start_time': Start_Time_KA,
    'End_time': End_Time_KA,
    'Total_duration': Total_Duration_KA,
    'Price': Prices_KA,
    "Seats_Available":Seats_Available_KA,
    "Ratings":Ratings_KA,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_10= pd.DataFrame(data_10)
df_buses_10.to_csv('buses_data10.csv', index=False)
df_buses_10

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,05:50,10h 35m,INR 1000,7 Seats available,3.7\n337,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,07:30,10h 30m,INR 660,10 Seats available,4.6\n591,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,09:00,10h 30m,INR 660,12 Seats available,4.5\n491,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,AdIntrCity SmartBus,AC Sleeper (2+1),22:00,08:15,10h 15m,INR 805,18 Seats available,4.3\n62,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Ashray Amrutyog,A/C Sleeper (2+1),22:00,09:30,11h 30m,INR 799,24 Seats available,4.3\n557,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
228,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,02:30,09h 00m,INR 900,26 Seats available,3.7\n13,https://www.redbus.in/bus-tickets/goa-to-shimoga,Goa to Shivamogga
229,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,INR 400,17 Seats available,3.2,https://www.redbus.in/bus-tickets/calangute-go...,Calangute (goa) to Goa
230,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,08:00,04h 30m,INR 1350,9 Seats available,1.3\n5,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
231,Shirdi Xpress,A/C Sleeper (2+1),08:15,11:30,03h 15m,419,19 Seats available,1.7\n9,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa


In [43]:
import pandas as pd
import re
import numpy as np
# csv to dataframe
df_1=pd.read_csv("buses_data1.csv")
df_2=pd.read_csv("buses_data2.csv")
df_3=pd.read_csv("buses_data3.csv")
df_4=pd.read_csv("buses_data4.csv")
df_5=pd.read_csv("buses_data5.csv")
df_6=pd.read_csv("buses_data6.csv")
df_7=pd.read_csv("buses_data7.csv")
df_8=pd.read_csv("buses_data8.csv")
df_9=pd.read_csv("buses_data9.csv")
df_10=pd.read_csv("buses_data10.csv")

df_red =pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,
                    df_7,df_8,df_9,df_10],ignore_index=True)
df_red 

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,APSRTC - 27133,METRO LUXURY A/C,19:40,22:45,03h 05m,INR 383,22 Seats available,4.4\n71,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
1,APSRTC - 27093,METRO LUXURY A/C,20:20,00:10,03h 50m,INR 383,28 Seats available,4.0\n27,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
2,APSRTC - 2217,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:30,00:30,04h 00m,INR 281,34 Seats available,3.5\n9,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
3,APSRTC - 3104,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",21:00,01:22,04h 22m,INR 281,29 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
4,APSRTC - 3189,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",21:20,01:15,03h 55m,INR 269,35 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
...,...,...,...,...,...,...,...,...,...,...
1710,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,02:30,09h 00m,INR 900,26 Seats available,3.7\n13,https://www.redbus.in/bus-tickets/goa-to-shimoga,Goa to Shivamogga
1711,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,INR 400,17 Seats available,3.2,https://www.redbus.in/bus-tickets/calangute-go...,Calangute (goa) to Goa
1712,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,08:00,04h 30m,INR 1350,9 Seats available,1.3\n5,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
1713,Shirdi Xpress,A/C Sleeper (2+1),08:15,11:30,03h 15m,419,19 Seats available,1.7\n9,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa


In [45]:
df_red.rename(columns={'Start_time': 'Departing Time'}, inplace=True)
df_red.rename(columns={'End_time': 'Reaching Time'}, inplace=True)
df_red.rename(columns={'Total_duration': 'Duration'}, inplace=True)
df_red.rename(columns={'Bus_name': 'Bus_Name'}, inplace=True)
df_red.rename(columns={'Bus_type': 'Bus_Type'}, inplace=True)
df_red.rename(columns={'Route_link': 'Route_Link'}, inplace=True)
df_red.rename(columns={'Route_name': 'Route_Name'}, inplace=True)


In [47]:
df_red['Price'] = [float(re.sub(r'INR\s*', '', prices)) for prices in df_red['Price']]
df_red['Ratings'] = df_red['Ratings'].astype(str).str.replace('NEW', '0', case=False)
df_red['Ratings'] = df_red['Ratings'].apply(lambda x: re.match(r'^\d+\.\d+', x).group(0) if re.match(r'^\d+\.\d+', x) else x)
#df_red = df_red[df_red['Ratings'].str.strip() != '']
df_red['Ratings'] = df_red['Ratings'].astype(float)
df_red


C:\Users\DELL\AppData\Local\Temp\ipykernel_22596\3262703634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_red['Ratings'] = df_red['Ratings'].astype(float)


,Bus_Name,Bus_Type,Departing Time,Reaching Time,Duration,Price,Seats_Available,Ratings,Route_Link,Route_Name
0,APSRTC - 27133,METRO LUXURY A/C,19:40,22:45,03h 05m,383.0,22 Seats available,4.4,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
1,APSRTC - 27093,METRO LUXURY A/C,20:20,00:10,03h 50m,383.0,28 Seats available,4.0,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
2,APSRTC - 2217,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:30,00:30,04h 00m,281.0,34 Seats available,3.5,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
3,APSRTC - 3104,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",21:00,01:22,04h 22m,281.0,29 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
4,APSRTC - 3189,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",21:20,01:15,03h 55m,269.0,35 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
...,...,...,...,...,...,...,...,...,...,...
1710,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,02:30,09h 00m,900.0,26 Seats available,3.7,https://www.redbus.in/bus-tickets/goa-to-shimoga,Goa to Shivamogga
1711,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,400.0,17 Seats available,3.2,https://www.redbus.in/bus-tickets/calangute-go...,Calangute (goa) to Goa
1712,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,08:00,04h 30m,1350.0,9 Seats available,1.3,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
1713,Shirdi Xpress,A/C Sleeper (2+1),08:15,11:30,03h 15m,419.0,19 Seats available,1.7,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa


In [49]:
import sqlite3
con = sqlite3.connect("Desktop/project/redbus.db")
cur = con.cursor()

In [57]:
import pandas as pd
con = sqlite3.connect("Desktop/project/redbus.db")
df_red.to_sql('redbus', con, if_exists='replace', index=False)
redbus_data = pd.read_sql_query('SELECT * FROM redbus', con)
redbus_data




,Bus_Name,Bus_Type,Departing Time,Reaching Time,Duration,Price,Seats_Available,Ratings,Route_Link,Route_Name
0,APSRTC - 27133,METRO LUXURY A/C,19:40,22:45,03h 05m,383.0,22 Seats available,4.4,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
1,APSRTC - 27093,METRO LUXURY A/C,20:20,00:10,03h 50m,383.0,28 Seats available,4.0,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
2,APSRTC - 2217,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:30,00:30,04h 00m,281.0,34 Seats available,3.5,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
3,APSRTC - 3104,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",21:00,01:22,04h 22m,281.0,29 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
4,APSRTC - 3189,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",21:20,01:15,03h 55m,269.0,35 Seats available,3.8,https://www.redbus.in/bus-tickets/kakinada-to-...,Kakinada to Visakhapatnam
...,...,...,...,...,...,...,...,...,...,...
1703,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,02:30,09h 00m,900.0,26 Seats available,3.7,https://www.redbus.in/bus-tickets/goa-to-shimoga,Goa to Shivamogga
1704,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,400.0,17 Seats available,3.2,https://www.redbus.in/bus-tickets/calangute-go...,Calangute (goa) to Goa
1705,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,08:00,04h 30m,1350.0,9 Seats available,1.3,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
1706,Shirdi Xpress,A/C Sleeper (2+1),08:15,11:30,03h 15m,419.0,19 Seats available,1.7,https://www.redbus.in/bus-tickets/belagavi-to-goa,Belagavi to Goa
